In [1]:
import os.path
import pandas as pd
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD

In [6]:
data = pd.read_csv("NONen_interim_01_05_ALL_IN_with_full_titles_translated_.csv")
data = data.applymap(str)
doc = data.translated_title

/Users/sagnikchakraborty/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
news_df = pd.DataFrame({'document':doc})

In [9]:
data.head()

,Unnamed: 0,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,...,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title,text,translated_title
0,0,43,896053216,19200101,192001,1920,1920.0027,nan,nan,nan,...,IN07,17911.0,28.6,77.2,-2106102,20200101001500,https://www.anandabazar.com/national/every-pol...,every police bullet is aimed at the constituti...,\n\n\n\nEvery police bullet is aimed at the Co...,every police bullet is aimed at the constituti...
1,1,137,896053310,19200101,192001,1920,1920.0027,COP,POLICE,nan,...,IN16,70179.0,18.7333,76.3833,-2088642,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,impersonation bans a police officer,Please enable javascript.aurangabad News ...,impersonation bans a police officer
2,2,202,896053375,19200101,192001,1920,1920.0027,EDU,STUDENT,nan,...,IN16,70184.0,18.975,72.8258,-2092174,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,welcome to the new year saying write date,Please enable javascript.mumbai news News ...,welcome to the new year saying write date
3,3,212,896053385,19200101,192001,1920,1920.0027,FRA,FRANCE,FRA,...,IN28,70294.0,22.5697,88.3697,-2092511,20200101001500,https://www.anandabazar.com/editorial/end-of-p...,end of participatory democracy right wing popu...,"\n\n\n\nEnd of participatory democracy, Right ...",end of participatory democracy right wing popu...
4,4,223,896053396,19200101,192001,1920,1920.0027,GOV,PRESIDENT,nan,...,IN16,70178.0,18.5333,73.8667,-2108361,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,bjps retreat due to their words,Please enable javascript.ahmednagar news News ...,bjps retreat due to their words


In [10]:
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [11]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sagnikchakraborty/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
print(stop_words)
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [13]:
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])


In [14]:
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [15]:
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])

X.shape

(62428, 1000)

In [91]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

lsa=svd_model.fit_transform(X)

len(svd_model.components_)

20

In [92]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

Topic 0: 
news
 
india
 
hindi
 
coronavirus
 
corona
 
delhi
 
lockdown
 
Topic 1: 
india
 
coronavirus
 
corona
 
lockdown
 
covid
 
virus
 
trump
 
Topic 2: 
india
 
news
 
international
 
trump
 
visit
 
donald
 
pakistan
 
Topic 3: 
coronavirus
 
lockdown
 
news
 
covid
 
updates
 
live
 
outbreak
 
Topic 4: 
delhi
 
violence
 
police
 
story
 
election
 
court
 
case
 
Topic 5: 
business
 
standard
 
government
 
indian
 
modi
 
trump
 
industry
 
Topic 6: 
modi
 
government
 
story
 
says
 
minister
 
narendra
 
national
 
Topic 7: 
story
 
case
 
police
 
court
 
nirbhaya
 
arrested
 
covid
 
Topic 8: 
case
 
police
 
court
 
government
 
national
 
nirbhaya
 
arrested
 
Topic 9: 
lockdown
 
police
 
covid
 
indian
 
people
 
arrested
 
mumbai
 
Topic 10: 
indian
 
trump
 
police
 
covid
 
donald
 
world
 
modi
 
Topic 11: 
covid
 
government
 
indian
 
cases
 
pradesh
 
delhi
 
madhya
 
Topic 12: 
covid
 
modi
 
case
 
lockdown
 
narendra
 
trump
 
national
 
Topic 13: 
nation

In [25]:
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = [["topic_1", "topic_2","topic_3", "topic_4","topic_5", "topic_6","topic_7", "topic_8","topic_9", "topic_10","topic_11", "topic_12","topic_13", "topic_14","topic_15", "topic_16","topic_17", "topic_18","topic_19", "topic_20"]])


In [26]:
topic_encoded_df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,0.0244904557823923,0.0124229401971120,-0.0497895722512074,-0.0237540750816023,0.0542810094741952,0.0004845178604001,-0.0191571865982112,0.0391062557013995,0.1074011757819068,0.1316069722588970,0.0361629720566719,-0.0837354383528653,-0.0416627549424738,-0.0308984445010094,-0.1162119637599232,0.0472887860099588,-0.0204530752591076,-0.0210509682454412,-0.0496843945034943,-0.0087446841879666
1,0.0361462103544639,0.0174019579525394,-0.0804149078811552,-0.0427945757426893,0.0972593427504055,0.0000096752575702,-0.0690588139107963,0.0802466909419220,0.2013157994182004,0.2372514289744703,0.0815803193170677,-0.1487992535346999,-0.0811377585131478,-0.0531662995486022,-0.2312955954697025,0.1044684065785997,-0.0365055031192073,-0.0357793932103845,-0.0896980756449504,-0.0181784511196779
2,0.0090176622139855,0.0069181093643148,-0.0075332191863199,-0.0022742889030613,0.0068139801816756,0.0004096906764575,0.0104541370977804,0.0088022199477096,0.0062656971785816,0.0000799480701897,0.0070157508317972,-0.0052368902764273,0.0005581342793645,-0.0037496932183286,-0.0051316533821481,0.0000454712009141,0.0053064636339827,0.0039709093366329,0.0067014746314214,0.0018403549724907
3,0.0154988615117915,0.0155007423456288,-0.0133635604953541,0.0069639421657556,-0.0040457825784952,0.0013929798083856,0.0087251104166217,-0.0017106373195926,-0.0074026499442111,-0.0075516847110950,0.0462824797312000,0.0079314762609220,0.0076023372365593,0.0062478785207037,-0.0134266747433028,-0.0029277652692214,-0.0212562941221277,0.0117023999847140,0.0472134414060615,-0.1007717126439521
4,0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,-0.0000000000000001,0.0000000000000021,-0.0000000000000080,-0.0000000000000006,0.0000000000000017,0.0000000000000014,0.0000000000000008,-0.0000000000000017,-0.0000000000000039,0.0000000000000100,-0.0000000000000033,0.0000000000000020,-0.0000000000000003,0.0000000000000001,0.0000000000000099


In [29]:
topic_encoded_df["documents"] = news_df['clean_doc']
display(topic_encoded_df[["documents", "topic_1", "topic_2","topic_3", "topic_4","topic_5", "topic_6","topic_7", "topic_8","topic_9", "topic_10","topic_11", "topic_12","topic_13", "topic_14","topic_15", "topic_16","topic_17", "topic_18","topic_19", "topic_20"]])

,documents,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,every police bullet aimed constitution wrote c...,0.0244904557823923,0.0124229401971120,-0.0497895722512074,-0.0237540750816023,0.0542810094741952,0.0004845178604001,-0.0191571865982112,0.0391062557013995,0.1074011757819068,...,0.0361629720566719,-0.0837354383528653,-0.0416627549424738,-0.0308984445010094,-0.1162119637599232,0.0472887860099588,-0.0204530752591076,-0.0210509682454412,-0.0496843945034943,-0.0087446841879666
1,impersonation bans police officer,0.0361462103544639,0.0174019579525394,-0.0804149078811552,-0.0427945757426893,0.0972593427504055,0.0000096752575702,-0.0690588139107963,0.0802466909419220,0.2013157994182004,...,0.0815803193170677,-0.1487992535346999,-0.0811377585131478,-0.0531662995486022,-0.2312955954697025,0.1044684065785997,-0.0365055031192073,-0.0357793932103845,-0.0896980756449504,-0.0181784511196779
2,welcome year saying write date,0.0090176622139855,0.0069181093643148,-0.0075332191863199,-0.0022742889030613,0.0068139801816756,0.0004096906764575,0.0104541370977804,0.0088022199477096,0.0062656971785816,...,0.0070157508317972,-0.0052368902764273,0.0005581342793645,-0.0037496932183286,-0.0051316533821481,0.0000454712009141,0.0053064636339827,0.0039709093366329,0.0067014746314214,0.0018403549724907
3,participatory democracy right wing populism ta...,0.0154988615117915,0.0155007423456288,-0.0133635604953541,0.0069639421657556,-0.0040457825784952,0.0013929798083856,0.0087251104166217,-0.0017106373195926,-0.0074026499442111,...,0.0462824797312000,0.0079314762609220,0.0076023372365593,0.0062478785207037,-0.0134266747433028,-0.0029277652692214,-0.0212562941221277,0.0117023999847140,0.0472134414060615,-0.1007717126439521
4,bjps retreat words,0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,-0.0000000000000001,0.0000000000000021,-0.0000000000000080,-0.0000000000000006,...,0.0000000000000014,0.0000000000000008,-0.0000000000000017,-0.0000000000000039,0.0000000000000100,-0.0000000000000033,0.0000000000000020,-0.0000000000000003,0.0000000000000001,0.0000000000000099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62423,sujata kotamuraju teaching india culture kenya...,0.2520719408246668,0.4666382701460494,0.2512190764209138,-0.0652933709771202,0.0509996712396413,-0.0047303471961395,-0.0031819282859124,0.0177118360822980,0.0453052975319214,...,-0.0420409466571171,-0.0467023002019815,-0.0406319867716101,0.0096669855206542,-0.0004084735479058,-0.0106652794810050,-0.0913833335787970,0.1213253633923314,0.7378619324061076,-0.0146839033364920
62424,airlines india resumed domestic flights despit...,0.1145612305841875,0.1952538291504025,0.0528304031345402,0.0004060045684710,-0.0237968901603376,-0.0029834122094649,-0.0153499870320373,0.0223464118939189,0.0038837104479747,...,0.0136331309404455,0.2037275098888109,0.1657652441877743,0.0095233014641508,-0.0829052615949570,0.0157523870064786,-0.0039963915863306,-0.0124997543863189,-0.0112123152244295,0.0018492246816332
62425,fitr prayers health well,0.0210878115314047,0.0185551668622430,-0.0396633649025980,0.0035762519302890,-0.0216540248414400,-0.0000769725639180,0.0149449331225393,-0.0068479830007040,-0.0013528526401479,...,0.0170535766390955,0.0163994204784708,-0.0077455817913485,0.0019507764653558,-0.0221351117334262,-0.0452905344654580,-0.0488476811321026,0.0215888444681103,-0.0009214057834654,0.0430079141837604
62426,,0.0000000000000000,0.0000000000000000,-0.0000000000000000,-0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,...,-0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000


In [31]:
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1", "topic_2","topic_3", "topic_4","topic_5", "topic_6","topic_7", "topic_8","topic_9", "topic_10","topic_11", "topic_12","topic_13", "topic_14","topic_15", "topic_16","topic_17", "topic_18","topic_19", "topic_20"], columns = (terms)).T
encoding_matrix

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
able,0.0018732313025107,0.0008708346630489,-0.0026284897613043,-0.0003927782985436,-0.0002523439601567,0.0000040130822448,0.0029055230724561,0.0012091498938620,0.0003596326516697,0.0022519145450784,-0.0032705806534216,-0.0002090246097337,-0.0008342300388258,0.0006742504407984,0.0039847476611524,-0.0004401314443772,-0.0004987185273930,0.0008402344383146,0.0001783055011834,0.0006162146177828
abroad,0.0026440447978045,0.0026522509771840,-0.0034843163588652,0.0006084253865364,-0.0015124258541554,0.0002837389999877,0.0030003977162273,-0.0001271838422647,0.0008251037858542,0.0026326289629962,0.0025957604771660,0.0012610489308851,-0.0007276795973903,0.0006695247865285,0.0029955291833605,-0.0029352801751294,-0.0030681750972168,0.0024010836884404,0.0061684889860575,-0.0004565051591040
accident,0.0051068932949755,0.0004706163382288,-0.0039262807354758,-0.0003196595453177,0.0017397962609621,-0.0000216706289252,0.0049305459693304,0.0036021900372431,0.0082461825613144,0.0081821410116916,0.0037563594471844,-0.0018134273030417,-0.0045511243212361,-0.0010889757098810,0.0007896091315884,-0.0033363402046625,0.0031647269486357,-0.0011425504463502,0.0091355969237245,0.0032431181425337
account,0.0027371239478680,0.0003808821195382,-0.0029773000502433,-0.0008856674323821,0.0010304261936817,0.0000835021623409,0.0075512384803139,0.0003307275815328,0.0002623061410949,0.0007861766647066,-0.0003296978270684,-0.0002668662259937,0.0000037672531703,0.0018448423174030,0.0001251877545114,0.0020423587001719,-0.0022313239588718,-0.0028337704875827,0.0012031945221046,0.0004432517039606
accounts,0.0022716300012204,0.0001045417778711,-0.0018697224075059,-0.0007373093026039,0.0008553914951024,0.0000593279154336,0.0051043002264576,-0.0011874634907542,0.0004875570704066,-0.0001282310056026,-0.0000330355374908,0.0006588755024384,-0.0022771324236092,-0.0018121616225990,-0.0000267237597711,0.0022773455775498,-0.0008093775031219,-0.0025953265967187,0.0022605119047089,-0.0010548366785501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yediyurappa,0.0012175108021920,0.0014489633274739,-0.0029615782637788,0.0013804728053705,0.0010391636978232,0.0000997634689047,0.0062028889953686,-0.0017978107793358,-0.0003676488476628,0.0006674823806620,-0.0004226284371811,-0.0007214933068881,0.0010746524473404,0.0001202046133023,-0.0012018848943433,-0.0070377587115365,-0.0074222197281444,0.0047915119531820,-0.0031861852501303,0.0000730282455462
yogi,0.0139113986216359,0.0014661700859676,-0.0189071850819515,-0.0048052227721530,0.0040648756600094,0.0004781355234441,0.0357371764870287,-0.0104826209672832,0.0204088193305342,0.0068386005892223,-0.0266686618560859,0.0199056332271916,-0.0258426567968278,-0.0103821304293287,-0.0051062571113267,-0.0073593257168509,-0.0036832875113435,0.0063981740851164,-0.0034322029159796,0.0228606743885277
young,0.0032589548222615,0.0025290294720855,-0.0034273879783906,-0.0019329787169811,0.0009096039072198,-0.0000895244460405,-0.0002099012413729,0.0037239579539628,0.0059320505520002,0.0043133696371321,0.0065398413753438,-0.0021246367653874,-0.0026891396789538,-0.0008857944538550,-0.0020731353404627,-0.0013609613515279,-0.0000636273021700,-0.0011062327910376,0.0091333749255653,0.0007572892062261
youth,0.0062286972453366,0.0020708724249600,-0.0065718490807003,-0.0016434538149943,0.0039382016288545,-0.0001043850690485,0.0028266770363807,0.0024751730422864,0.0066938384844173,0.0032617760558345,0.0013416500794652,-0.0083500672081444,-0.0021178519005117,0.0006722280979536,-0.0078454426589509,0.0006299925801590,-0.0004393482715791,-0.0031809624983596,0.0019937701628265,-0.0047169269864993


In [32]:
encoding_matrix.to_csv('topics.csv',encoding='utf-8')

In [44]:
import numpy as np
def filter_title(row):

    list1=[row['topic_1'].iloc[0],row['topic_2'].iloc[0],row['topic_3'].iloc[0],row['topic_4'].iloc[0],row['topic_5'].iloc[0],row['topic_6'].iloc[0],row['topic_7'].iloc[0],row['topic_8'].iloc[0],row['topic_9'].iloc[0],row['topic_10'].iloc[0],row['topic_11'].iloc[0],row['topic_12'].iloc[0],row['topic_13'].iloc[0],row['topic_14'].iloc[0],row['topic_15'].iloc[0],row['topic_16'].iloc[0],row['topic_17'].iloc[0],row['topic_18'].iloc[0],row['topic_19'].iloc[0],row['topic_20'].iloc[0]]
    list1.sort()
    return list1[-1]

topic_encoded_df['Dominant_Topic']=topic_encoded_df.apply(lambda row : filter_title(row),axis=1)

In [54]:
topic_encoded_df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,documents,Dominant_Topic
0,0.0244904557823923,0.0124229401971120,-0.0497895722512074,-0.0237540750816023,0.0542810094741952,0.0004845178604001,-0.0191571865982112,0.0391062557013995,0.1074011757819068,0.1316069722588970,...,-0.0416627549424738,-0.0308984445010094,-0.1162119637599232,0.0472887860099588,-0.0204530752591076,-0.0210509682454412,-0.0496843945034943,-0.0087446841879666,every police bullet aimed constitution wrote c...,0.1316069722588970
1,0.0361462103544639,0.0174019579525394,-0.0804149078811552,-0.0427945757426893,0.0972593427504055,0.0000096752575702,-0.0690588139107963,0.0802466909419220,0.2013157994182004,0.2372514289744703,...,-0.0811377585131478,-0.0531662995486022,-0.2312955954697025,0.1044684065785997,-0.0365055031192073,-0.0357793932103845,-0.0896980756449504,-0.0181784511196779,impersonation bans police officer,0.2372514289744703
2,0.0090176622139855,0.0069181093643148,-0.0075332191863199,-0.0022742889030613,0.0068139801816756,0.0004096906764575,0.0104541370977804,0.0088022199477096,0.0062656971785816,0.0000799480701897,...,0.0005581342793645,-0.0037496932183286,-0.0051316533821481,0.0000454712009141,0.0053064636339827,0.0039709093366329,0.0067014746314214,0.0018403549724907,welcome year saying write date,0.0104541370977804
3,0.0154988615117915,0.0155007423456288,-0.0133635604953541,0.0069639421657556,-0.0040457825784952,0.0013929798083856,0.0087251104166217,-0.0017106373195926,-0.0074026499442111,-0.0075516847110950,...,0.0076023372365593,0.0062478785207037,-0.0134266747433028,-0.0029277652692214,-0.0212562941221277,0.0117023999847140,0.0472134414060615,-0.1007717126439521,participatory democracy right wing populism ta...,0.0472134414060615
4,0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,-0.0000000000000001,0.0000000000000021,-0.0000000000000080,-0.0000000000000006,0.0000000000000017,...,-0.0000000000000017,-0.0000000000000039,0.0000000000000100,-0.0000000000000033,0.0000000000000020,-0.0000000000000003,0.0000000000000001,0.0000000000000099,bjps retreat words,0.0000000000000100


In [53]:
topic_encoded_df.drop(['Topic'],axis=1,inplace=True)

/Users/sagnikchakraborty/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [56]:

def filter_topic(row):

    if row['Dominant_Topic'].iloc[0]==row['topic_1'].iloc[0]:
        output='1'
        
    elif row['Dominant_Topic'].iloc[0]==row['topic_2'].iloc[0]:
        output='2'
    elif row['Dominant_Topic'].iloc[0]==row['topic_3'].iloc[0]:
        output='3'
    elif row['Dominant_Topic'].iloc[0]==row['topic_4'].iloc[0]:
        output='4'     
    elif row['Dominant_Topic'].iloc[0]==row['topic_5'].iloc[0]:
        output='5'
    elif row['Dominant_Topic'].iloc[0]==row['topic_6'].iloc[0]:
        output='6'
    elif row['Dominant_Topic'].iloc[0]==row['topic_7'].iloc[0]:
        output='7'
    elif row['Dominant_Topic'].iloc[0]==row['topic_8'].iloc[0]:
        output='8'
    elif row['Dominant_Topic'].iloc[0]==row['topic_9'].iloc[0]:
        output='9'
    elif row['Dominant_Topic'].iloc[0]==row['topic_10'].iloc[0]:
        output='10'
    elif row['Dominant_Topic'].iloc[0]==row['topic_11'].iloc[0]:
        output='11'
    elif row['Dominant_Topic'].iloc[0]==row['topic_12'].iloc[0]:
        output='12'
    elif row['Dominant_Topic'].iloc[0]==row['topic_13'].iloc[0]:
        output='13'
    elif row['Dominant_Topic'].iloc[0]==row['topic_14'].iloc[0]:
        output='14'
    elif row['Dominant_Topic'].iloc[0]==row['topic_15'].iloc[0]:
        output='15'
    elif row['Dominant_Topic'].iloc[0]==row['topic_16'].iloc[0]:
        output='16'
    elif row['Dominant_Topic'].iloc[0]==row['topic_17'].iloc[0]:
        output='17'
    elif row['Dominant_Topic'].iloc[0]==row['topic_18'].iloc[0]:
        output='18'
    elif row['Dominant_Topic'].iloc[0]==row['topic_19'].iloc[0]:
        output='19'
    elif row['Dominant_Topic'].iloc[0]==row['topic_20'].iloc[0]:
        output='20'
    
    return output

topic_encoded_df['Topic']=topic_encoded_df.apply(lambda row : filter_topic(row),axis=1)

In [55]:
data['clean_doc'] = data['translated_title'].str.replace("[^a-zA-Z#]", " ")
data['clean_doc'] = data['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data['clean_doc'] = data['clean_doc'].apply(lambda x: x.lower())
tokenized_doc = data['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

data['clean_doc'] = detokenized_doc


In [65]:
topic_encoded_df.head()

,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,...,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,documents,Dominant_Topic,Topic
0,0.0244904557823923,0.0124229401971120,-0.0497895722512074,-0.0237540750816023,0.0542810094741952,0.0004845178604001,-0.0191571865982112,0.0391062557013995,0.1074011757819068,0.1316069722588970,...,-0.0308984445010094,-0.1162119637599232,0.0472887860099588,-0.0204530752591076,-0.0210509682454412,-0.0496843945034943,-0.0087446841879666,every police bullet aimed constitution wrote c...,0.1316069722588970,10
1,0.0361462103544639,0.0174019579525394,-0.0804149078811552,-0.0427945757426893,0.0972593427504055,0.0000096752575702,-0.0690588139107963,0.0802466909419220,0.2013157994182004,0.2372514289744703,...,-0.0531662995486022,-0.2312955954697025,0.1044684065785997,-0.0365055031192073,-0.0357793932103845,-0.0896980756449504,-0.0181784511196779,impersonation bans police officer,0.2372514289744703,10
2,0.0090176622139855,0.0069181093643148,-0.0075332191863199,-0.0022742889030613,0.0068139801816756,0.0004096906764575,0.0104541370977804,0.0088022199477096,0.0062656971785816,0.0000799480701897,...,-0.0037496932183286,-0.0051316533821481,0.0000454712009141,0.0053064636339827,0.0039709093366329,0.0067014746314214,0.0018403549724907,welcome year saying write date,0.0104541370977804,7
3,0.0154988615117915,0.0155007423456288,-0.0133635604953541,0.0069639421657556,-0.0040457825784952,0.0013929798083856,0.0087251104166217,-0.0017106373195926,-0.0074026499442111,-0.0075516847110950,...,0.0062478785207037,-0.0134266747433028,-0.0029277652692214,-0.0212562941221277,0.0117023999847140,0.0472134414060615,-0.1007717126439521,participatory democracy right wing populism ta...,0.0472134414060615,19
4,0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,-0.0000000000000001,0.0000000000000021,-0.0000000000000080,-0.0000000000000006,0.0000000000000017,...,-0.0000000000000039,0.0000000000000100,-0.0000000000000033,0.0000000000000020,-0.0000000000000003,0.0000000000000001,0.0000000000000099,bjps retreat words,0.0000000000000100,15


In [63]:
topic_df=topic_encoded_df[['documents','Topic']]

In [66]:
final_df=pd.concat([data, topic_df], axis=1, join='inner')

In [77]:
final_df.columns

Index(['Unnamed: 0', 'index', 'GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year',
       'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode',
       'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code',
       'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code',
       'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode',
       'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code',
       'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code',
       'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'CAMEOCodeDescription',
       'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale',
       'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type',
       'Actor1Geo_FullName', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code',
       'Actor1Geo_ADM2Code', 'Actor1Geo_Lat', 'Actor1Geo_Long',
       'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName',
       'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Cod

In [75]:
final_df.rename(columns = {('documents',):'documents'}, inplace = True)

In [76]:
final_df.rename(columns = {('Topic',):'Topic'}, inplace = True)

In [108]:
final_df.to_csv('final_df_lsa.csv',encoding='utf-8')

In [109]:
filtered_final_df=final_df[(final_df['Topic']=='1') | (final_df['Topic']=='2') | (final_df['Topic']=='5') | (final_df['Topic']=='8') | (final_df['Topic']=='9') | (final_df['Topic']=='15') | (final_df['Topic']=='19')]

In [110]:
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(filtered_final_df['clean_doc'])

X.shape

(34052, 1000)

In [111]:
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

lsa=svd_model.fit_transform(X)

len(svd_model.components_)

20

In [112]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(",")

Topic 0: 
news
,
india
,
hindi
,
international
,
corona
,
delhi
,
coronavirus
,
Topic 1: 
india
,
coronavirus
,
corona
,
virus
,
lockdown
,
covid
,
delhi
,
Topic 2: 
corona
,
delhi
,
virus
,
story
,
violence
,
case
,
police
,
Topic 3: 
corona
,
virus
,
positive
,
news
,
cases
,
patients
,
patient
,
Topic 4: 
story
,
case
,
coronavirus
,
court
,
nirbhaya
,
lockdown
,
covid
,
Topic 5: 
case
,
court
,
nirbhaya
,
supreme
,
convicts
,
rape
,
high
,
Topic 6: 
coronavirus
,
people
,
lockdown
,
covid
,
updates
,
cases
,
live
,
Topic 7: 
international
,
coronavirus
,
people
,
lockdown
,
cases
,
updates
,
covid
,
Topic 8: 
people
,
arrested
,
police
,
hindi
,
india
,
killed
,
violence
,
Topic 9: 
world
,
bank
,
country
,
year
,
largest
,
economic
,
billion
,
Topic 10: 
arrested
,
police
,
case
,
murder
,
accused
,
violence
,
coronavirus
,
Topic 11: 
court
,
arrested
,
violence
,
supreme
,
high
,
police
,
mumbai
,
Topic 12: 
violence
,
country
,
people
,
news
,
latest
,
case
,
nirbhaya
,
Topic 13

In [123]:
filtered_final_df

,Unnamed: 0,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,...,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title,text,translated_title,clean_doc,documents,Topic
3,3,212,896053385,19200101,192001,1920,1920.0027,FRA,FRANCE,FRA,...,88.3697,-2092511,20200101001500,https://www.anandabazar.com/editorial/end-of-p...,end of participatory democracy right wing popu...,"\n\n\n\nEnd of participatory democracy, Right ...",end of participatory democracy right wing popu...,participatory democracy right wing populism ta...,participatory democracy right wing populism ta...,19
4,4,223,896053396,19200101,192001,1920,1920.0027,GOV,PRESIDENT,nan,...,73.8667,-2108361,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,bjps retreat due to their words,Please enable javascript.ahmednagar news News ...,bjps retreat due to their words,bjps retreat words,bjps retreat words,15
11,11,634,896056396,19200101,192001,1920,1920.0027,CVL,RESIDENTS,nan,...,88.3697,-2092511,20200101003000,https://g1.globo.com/mundo/noticia/2019/12/31/...,india tem ano novo com novos protestos contra ...,\n Índia tem Ano Novo com novos protestos cont...,India has a new year with new protests citizen...,india year protests citizenship,india year protests citizenship,2
12,12,853,896056615,19200101,192001,1920,1920.0027,MED,TELEVISION,nan,...,77.2167,-2094230,20200101003000,https://www.glasistre.hr/svijet/istocna-hemisf...,istocna hemisfera usla u novo desetljece uz va...,"\n\n\n\n\nAUSTRALIJA, KINA, JAPAN, INDIJA... I...",eastern hemisphere entered the new decade with...,eastern hemisphere entered decade fireworks,eastern hemisphere entered decade fireworks,15
14,14,1078,896058998,19200101,192001,1920,1920.0027,BUS,COMPANY,nan,...,73.15,-2099261,20200101004500,https://www.loksatta.com/thane-news/stalled-pr...,stalled projects in thane district will comple...,\n\n\n\n\n\n\n\n\n\n\n\n\t\t\tstalled projects...,stalled projects in thane district will comple...,stalled projects thane district complete,stalled projects thane district complete,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62420,62420,39400,925348446,20200524,202005,2020,2020.3945,BUS,LANDLORD,nan,...,85.9,-2094031,20200524224500,https://www.telemetro.com/ocio/2020/05/24/jove...,joven pedalea mas de mil kilometros hacia casa...,\n\n\n\n\n\nJoven pedalea más de mil kilómetro...,young pedaling over a thousand kilometers home...,young pedaling thousand kilometers homeward india,young pedaling thousand kilometers homeward india,2
62422,62422,40108,925351421,20200524,202005,2020,2020.3945,INDGOV,MAHARASHTRA,IND,...,72.8258,-2092174,20200524231500,https://www.prabhatkhabar.com/state/jharkhand/...,ranchi airport ready for airline irsa munda ai...,\n\n\n\n\n\nranchi airport ready for airline i...,ranchi airport ready for airline irsa munda ai...,ranchi airport ready airline irsa munda airpor...,ranchi airport ready airline irsa munda airpor...,1
62423,62423,40125,925351438,20200524,202005,2020,2020.3945,KEN,KENYA,KEN,...,81.1333,-2102931,20200524231500,https://www.sakshi.com/news/family/sujata-kota...,sujata kotamuraju teaching india culture kenya...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,sujata kotamuraju teaching india culture kenya...,sujata kotamuraju teaching india culture kenya...,sujata kotamuraju teaching india culture kenya...,19
62426,62426,40545,925354361,20200524,202005,2020,2020.3945,EUR,EUROPEAN,EUR,...,77.0,IN,20200524234500,https://www.alawan.org/2020/05/19/%D8%A3%D9%81...,A7 2,\n\n\n\n \n\n\n\n\n\n\nأفكار فيما وراء الكورون...,AhA,,,1


In [121]:
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df_new = pd.DataFrame(lsa, columns = [["topic_1", "topic_2","topic_3", "topic_4","topic_5", "topic_6","topic_7", "topic_8","topic_9", "topic_10","topic_11", "topic_12","topic_13", "topic_14","topic_15", "topic_16","topic_17", "topic_18","topic_19", "topic_20"]])



In [122]:
topic_encoded_df_new["documents"] = filtered_final_df['clean_doc']
display(topic_encoded_df_new[["documents", "topic_1", "topic_2","topic_3", "topic_4","topic_5", "topic_6","topic_7", "topic_8","topic_9", "topic_10","topic_11", "topic_12","topic_13", "topic_14","topic_15", "topic_16","topic_17", "topic_18","topic_19", "topic_20"]])

,documents,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,NaN,0.0088246325020377,0.0120355634840689,0.0076400612188017,0.0006696266955629,0.0022172658388466,-0.0023097381897267,0.0145590329445291,0.0044728972627870,0.0073763273282296,...,-0.0073596768220522,0.0048571689866069,-0.0056780033250989,-0.0138978130852674,0.0031488850006591,-0.0051925477003628,0.0102764925829345,-0.0061062424552441,0.0064472629143169,-0.0002202365813399
1,NaN,-0.0000000000000000,-0.0000000000000000,-0.0000000000000017,-0.0000000000000089,0.0000000000000023,-0.0000000000000016,0.0000000000000032,0.0000000000000011,-0.0000000000000002,...,-0.0000000000000004,-0.0000000000000004,0.0000000000000009,0.0000000000000004,0.0000000000000020,0.0000000000000039,-0.0000000000000026,0.0000000000000001,-0.0000000000000003,0.0000000000000006
2,NaN,0.0766589183326928,0.2540719333931733,-0.0754121461943765,-0.0340656503084476,-0.0085022607315955,0.0042578347665636,-0.0505115600624349,-0.0268820464552459,0.0576709671562874,...,0.0032358554481167,0.0190605663276707,-0.0143157953525629,0.0314364760871909,-0.0210878172027711,-0.0154489715566828,-0.0178788721220302,0.0618592775694327,0.0158384339111444,0.0501415227774740
3,participatory democracy right wing populism ta...,0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,0.0000000000000030,-0.0000000000000006,-0.0000000000000014,-0.0000000000000014,0.0000000000000008,...,-0.0000000000000000,-0.0000000000000001,-0.0000000000000000,0.0000000000000002,-0.0000000000000004,0.0000000000000017,-0.0000000000000021,0.0000000000000020,0.0000000000000010,0.0000000000000001
4,bjps retreat words,0.0093922803258611,0.0008472683331923,0.0115340497708144,0.0049324376498981,0.0038267595126427,0.0056691028715259,0.0081244907037909,-0.0029549838335521,-0.0020217736142132,...,0.0022210526385187,0.0130333888183753,-0.0080272444962672,0.0014989599185920,-0.0077105947459895,-0.0214348560317475,-0.0076195481327349,-0.0003573748618281,-0.0021141206124897,-0.0003574729796989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34047,kolhapur polices tweet comparing corona mask h...,0.0775480821534996,0.2721853199481105,-0.0826587802041213,-0.0122341901780286,-0.0090068316310935,0.0014395559701980,-0.0319307256364492,-0.0164131156276713,0.0513373139736125,...,-0.0061883631537872,-0.0057251196004454,-0.0085365147215053,0.0088672051810081,-0.0091140128632308,-0.0064980432819979,-0.0085451209709823,0.0154432510288412,-0.0051553877037944,0.0160006250080822
34048,NaN,0.3560324698671016,-0.1048657316017847,-0.0218218836524505,0.0012086669376983,-0.0072028744938380,0.0000499187739943,0.0208423277807358,-0.0938622621726309,0.0313857646538891,...,-0.0144456079516022,0.0124012484848500,-0.1035738989842148,-0.0919357699034772,0.0722613928238452,-0.1300702159085772,-0.1272665142220490,-0.0008948163686471,-0.0311955666794894,-0.0074920852930340
34049,NaN,0.1650299642891414,0.5478918730587852,-0.0882860370320022,-0.0327822024323671,0.0117855813441244,-0.0169022717410864,0.3292434795853450,0.1519962373467910,0.6194666012804348,...,-0.2478187551785626,-0.1140891589443525,0.0767194408809021,-0.0247191881004917,-0.0906689920753878,0.0852210206884873,0.0369779825945263,0.0173243901141209,-0.0141606145041664,0.0073162582512256
34050,kamal nath government floor test vidhan sabha ...,0.0000000000000000,0.0000000000000000,-0.0000000000000000,0.0000000000000000,-0.0000000000000000,0.0000000000000000,0.0000000000000000,0.0000000000000000,0.0000000000000000,...,0.0000000000000000,0.0000000000000000,0.0000000000000000,0.0000000000000000,-0.0000000000000000,0.0000000000000000,-0.0000000000000000,-0.0000000000000000,0.0000000000000000,-0.0000000000000000


In [124]:
topic_encoded_df_new['Dominant_Topic']=topic_encoded_df_new.apply(lambda row : filter_title(row),axis=1)

In [125]:
topic_encoded_df_new['Topic']=topic_encoded_df_new.apply(lambda row : filter_topic(row),axis=1)

In [126]:
topic_df_new=topic_encoded_df_new[['documents','Topic']]

In [127]:
final_df_new=pd.concat([filtered_final_df, topic_df_new], axis=1, join='inner')

In [129]:
final_df_new

,Unnamed: 0,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,...,DATEADDED,SOURCEURL,title,text,translated_title,clean_doc,documents,Topic,"(documents,)","(Topic,)"
3,3,212,896053385,19200101,192001,1920,1920.0027,FRA,FRANCE,FRA,...,20200101001500,https://www.anandabazar.com/editorial/end-of-p...,end of participatory democracy right wing popu...,"\n\n\n\nEnd of participatory democracy, Right ...",end of participatory democracy right wing popu...,participatory democracy right wing populism ta...,participatory democracy right wing populism ta...,19,participatory democracy right wing populism ta...,5
4,4,223,896053396,19200101,192001,1920,1920.0027,GOV,PRESIDENT,nan,...,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,bjps retreat due to their words,Please enable javascript.ahmednagar news News ...,bjps retreat due to their words,bjps retreat words,bjps retreat words,15,bjps retreat words,12
11,11,634,896056396,19200101,192001,1920,1920.0027,CVL,RESIDENTS,nan,...,20200101003000,https://g1.globo.com/mundo/noticia/2019/12/31/...,india tem ano novo com novos protestos contra ...,\n Índia tem Ano Novo com novos protestos cont...,India has a new year with new protests citizen...,india year protests citizenship,india year protests citizenship,2,india year protests citizenship,9
12,12,853,896056615,19200101,192001,1920,1920.0027,MED,TELEVISION,nan,...,20200101003000,https://www.glasistre.hr/svijet/istocna-hemisf...,istocna hemisfera usla u novo desetljece uz va...,"\n\n\n\n\nAUSTRALIJA, KINA, JAPAN, INDIJA... I...",eastern hemisphere entered the new decade with...,eastern hemisphere entered decade fireworks,eastern hemisphere entered decade fireworks,15,eastern hemisphere entered decade fireworks,10
14,14,1078,896058998,19200101,192001,1920,1920.0027,BUS,COMPANY,nan,...,20200101004500,https://www.loksatta.com/thane-news/stalled-pr...,stalled projects in thane district will comple...,\n\n\n\n\n\n\n\n\n\n\n\n\t\t\tstalled projects...,stalled projects in thane district will comple...,stalled projects thane district complete,stalled projects thane district complete,1,stalled projects thane district complete,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34044,34044,21050,912640655,20200316,202003,2020,2020.2082,CVL,VILLAGE,nan,...,20200316094500,https://navbharattimes.indiatimes.com/jokes/fa...,coronavirus update boy use corona mask as a sh...,Coronavirus Fake News: कोरोना मास्क पहनकर पापा...,coronavirus update boy use corona mask as a sh...,coronavirus update corona mask shield cigarett...,coronavirus update corona mask shield cigarett...,2,coronavirus update corona mask shield cigarett...,2
34045,34045,21184,912640789,20200316,202003,2020,2020.2082,GOV,BUREAUCRAT,nan,...,20200316094500,https://jatim.sindonews.com/read/26611/1/coron...,corona mewabah pemkab mojokerto tak keluarkan ...,"Corona Mewabah, Pemkab Mojokerto Tak Keluar...",Mojokerto regency endemic corona not remove pr...,mojokerto regency endemic corona remove prefer...,mojokerto regency endemic corona remove prefer...,1,mojokerto regency endemic corona remove prefer...,5
34046,34046,21400,912641005,20200316,202003,2020,2020.2082,LEG,LEGISLATIVE ASSEMBLY,nan,...,20200316094500,https://www.dinamani.com/tamilnadu/2020/mar/16...,visitors are not allowed in tamil nadu assembl...,\n\n\n\nEnable Javscript for better performanc...,visitors are not allowed in tamil nadu assembl...,visitors allowed tamil nadu assembly,visitors allowed tamil nadu assembly,5,visitors allowed tamil nadu assembly,9
34047,34047,21828,912642731,20200316,202003,2020,2020.2082,COP,POLICE,nan,...,20200316100000,https://maharashtratimes.indiatimes.com/mahara...,kolhapur polices tweet comparing corona mask a...,Please enable javascript.Kolhapur Police's Twe...,kolhapur polices tweet comparing corona mask a...,kolhapur polices tweet comparing corona mask h...,kolhapur polices tweet comparing corona mask h...,1,

In [131]:
final_df_new.rename(columns = {('Topic',):'Topic_new'}, inplace = True)
final_df_new.rename(columns = {('documents',):'documents_new'}, inplace = True)

In [132]:
final_df_new.to_csv('final_df_lsa2.csv',encoding='utf-8')

In [146]:
final_df_new[final_df_new['Topic_new']!='10']

,Unnamed: 0,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,...,DATEADDED,SOURCEURL,title,text,translated_title,clean_doc,documents,Topic,documents_new,Topic_new
3,3,212,896053385,19200101,192001,1920,1920.0027,FRA,FRANCE,FRA,...,20200101001500,https://www.anandabazar.com/editorial/end-of-p...,end of participatory democracy right wing popu...,"\n\n\n\nEnd of participatory democracy, Right ...",end of participatory democracy right wing popu...,participatory democracy right wing populism ta...,participatory democracy right wing populism ta...,19,participatory democracy right wing populism ta...,5
4,4,223,896053396,19200101,192001,1920,1920.0027,GOV,PRESIDENT,nan,...,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,bjps retreat due to their words,Please enable javascript.ahmednagar news News ...,bjps retreat due to their words,bjps retreat words,bjps retreat words,15,bjps retreat words,12
11,11,634,896056396,19200101,192001,1920,1920.0027,CVL,RESIDENTS,nan,...,20200101003000,https://g1.globo.com/mundo/noticia/2019/12/31/...,india tem ano novo com novos protestos contra ...,\n Índia tem Ano Novo com novos protestos cont...,India has a new year with new protests citizen...,india year protests citizenship,india year protests citizenship,2,india year protests citizenship,9
20,20,1282,896059202,19200101,192001,1920,1920.0027,IND,MUMBAI,IND,...,20200101004500,https://www.loksatta.com/arthasatta-news/nse-s...,nse star mf mumbai share market mutual fund ak...,\n\n\n\n\n\n\n\n\n\n\n\n\t\t\tNSE Star mf Mumb...,nse star mf mumbai share market mutual fund ak...,star mumbai share market mutual fund,star mumbai share market mutual fund,9,star mumbai share market mutual fund,3
30,30,4459,896070904,19200101,192001,1920,1920.0027,nan,nan,nan,...,20200101023000,https://www.sakshi.com/news/telangana/major-ev...,major events 1st january 2020 1252360,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,major events 1st january 2020 1252360,major events january,major events january,8,major events january,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34044,34044,21050,912640655,20200316,202003,2020,2020.2082,CVL,VILLAGE,nan,...,20200316094500,https://navbharattimes.indiatimes.com/jokes/fa...,coronavirus update boy use corona mask as a sh...,Coronavirus Fake News: कोरोना मास्क पहनकर पापा...,coronavirus update boy use corona mask as a sh...,coronavirus update corona mask shield cigarett...,coronavirus update corona mask shield cigarett...,2,coronavirus update corona mask shield cigarett...,2
34045,34045,21184,912640789,20200316,202003,2020,2020.2082,GOV,BUREAUCRAT,nan,...,20200316094500,https://jatim.sindonews.com/read/26611/1/coron...,corona mewabah pemkab mojokerto tak keluarkan ...,"Corona Mewabah, Pemkab Mojokerto Tak Keluar...",Mojokerto regency endemic corona not remove pr...,mojokerto regency endemic corona remove prefer...,mojokerto regency endemic corona remove prefer...,1,mojokerto regency endemic corona remove prefer...,5
34046,34046,21400,912641005,20200316,202003,2020,2020.2082,LEG,LEGISLATIVE ASSEMBLY,nan,...,20200316094500,https://www.dinamani.com/tamilnadu/2020/mar/16...,visitors are not allowed in tamil nadu assembl...,\n\n\n\nEnable Javscript for better performanc...,visitors are not allowed in tamil nadu assembl...,visitors allowed tamil nadu assembly,visitors allowed tamil nadu assembly,5,visitors allowed tamil nadu assembly,9
34047,34047,21828,912642731,20200316,202003,2020,2020.2082,COP,POLICE,nan,...,20200316100000,https://maharashtratimes.indiatimes.com/mahara...,kolhapur polices tweet comparing corona mask a...,Please enable javascript.Kolhapur Police's Twe...,kolhapur polices tweet comparing corona mask a...,kolhapur polices tweet comparing corona mask h...,kolhapur polices tweet comparing corona mask h...,1,kolhapur polices tweet comparing corona mask h...,2


In [147]:
filtered_final_df_new=final_df_new[(final_df_new['Topic_new']!='4') & (final_df_new['Topic_new']!='6') & (final_df_new['Topic_new']!='10') & (final_df_new['Topic_new']!='14') & (final_df_new['Topic_new']!='16')]




In [149]:
filtered_final_df_new.to_csv('NonEn_filtered_LSA.csv',encoding='utf-8')